## ETHZ-03-02-01 Map of b0 changes - Sentinel-1

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype.

### Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

### <a name="objective">Objective 




### <a name="data">Data

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'ETHZ-03-02-01 Map of b0 changes'),
                ('abstract', 'This application takes a pair of Sentinel-1 products and generates a map of b0 changes'),
                ('id', 'ewf-ethz-03-02-01')])

### Parameter Definition 

Output file format:

* BEAM-DIMAP
* GeoTIFF-BigTIFF

In [ ]:
format = dict([('id', 'format'),
               ('value', 'GeoTIFF-BigTIFF'),
               ('title', 'Output file format'),
               ('abstract', 'Output file format: BEAM-DIMAP or GeoTIFF-BigTIFF')])

### <a name="runtime">Runtime parameter definition
    
The variable values in this section are only relevant for the basic test case. In an actual processing context, the values are replaced by those of the parameters for the process execution.

**Input identifiers**

This is the Sentinel-1 stack of master and slave products' identifiers:

In [ ]:
#input_identifiers = ('S1A_IW_SLC__1SDV_20161030T163141_20161030T163208_013722_01603F_4094', 'S1A_IW_SLC__1SDV_20161018T163141_20161018T163208_013547_015AEB_5994')

input_identifiers = ('S1A_IW_SLC__1SDV_20161107T170553_20161107T170620_013839_0163F7_A66D', 'S1A_IW_SLC__1SDV_20161026T170553_20161026T170620_013664_015E7F_BD83')

**Input references**

This is the Sentinel-1 stack of catalogue references:

In [ ]:
#input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161030T163141_20161030T163208_013722_01603F_4094', 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161018T163141_20161018T163208_013547_015AEB_5994')

input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161107T170553_20161107T170620_013839_0163F7_A66D','https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161026T170553_20161026T170620_013664_015E7F_BD83')

**Data path**

This path defines where the data is staged-in:

In [ ]:
data_path = "/workspace/Better_3rd_phase/Applications/ETHZ-03-02-01/temp/data"

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import cioppy

import gdal
import osr

from shapely.wkt import loads
from shapely.geometry import box

import lxml.etree as etree
import numpy as np
import os

import shutil

In [ ]:
import lxml.etree as etree
import subprocess
import tempfile
import time
import psutil
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap
import os

class GraphProcessor():
    
    def __init__(self):
        
        self.root = etree.Element('graph')
    
        version = etree.SubElement(self.root, 'version')
        version.text = '1.0'
        self.pid = None
        self.p = None
   
    def view_graph(self):
        
        print etree.tostring(self.root , pretty_print=True)
        
    def add_node(self, node_id, operator, parameters, source):
    
        xpath_expr = '/graph/node[@id="%s"]' % node_id

        if len(self.root.xpath(xpath_expr)) != 0:

            node_elem = self.root.xpath(xpath_expr)[0]
            operator_elem = self.root.xpath(xpath_expr + '/operator')[0]
            sources_elem = self.root.xpath(xpath_expr + '/sources')[0]
            parameters_elem = self.root.xpath(xpath_expr + '/parameters')

            for key, value in parameters.iteritems():
                p_elem = self.root.xpath(xpath_expr + '/parameters/%s' % key)[0]
                p_elem.text = value
        else:

            node_elem = etree.SubElement(self.root, 'node')
            operator_elem = etree.SubElement(node_elem, 'operator')
            sources_elem = etree.SubElement(node_elem, 'sources')

            if isinstance(source, list):

                for index, s in enumerate(source):
                    if index == 0:  
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')

                    else: 
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct.%s' % str(index))

                    source_product_elem.attrib['refid'] = s

            elif source != '':
                source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')
                source_product_elem.attrib['refid'] = source

            parameters_elem = etree.SubElement(node_elem, 'parameters')
            parameters_elem.attrib['class'] = 'com.bc.ceres.binding.dom.XppDomElement'

            for key, value in parameters.iteritems():

                # here I have to adapt the code
                
                if operator == 'BandMaths':
                    
                    if isinstance(value, dict):
                        
                        parameter_elem = etree.SubElement(parameters_elem, key)
                        
                        for key2, value2 in value.iteritems():
                            parameter_elem2 = etree.SubElement(parameter_elem, key2)
                            #parameter_elem.text = value
                            if isinstance(value2, dict):
                                for key3, value3 in value2.iteritems():
                                    parameter_elem3 = etree.SubElement(parameter_elem2, key3)
                                    parameter_elem3.text = value3
                            pass
                    
                    else:
                        parameter_elem = etree.SubElement(parameters_elem, key)
                        parameter_elem.text = value
                else:
                    parameter_elem = etree.SubElement(parameters_elem, key)
                    parameter_elem.text = value
                    
                    
        
        node_elem.attrib['id'] = node_id

        operator_elem.text = operator 

    def save_graph(self, filename):
        
        with open(filename, 'wb') as file:
            file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
            file.write(etree.tostring(self.root, pretty_print=True))
     
    def plot_graph(self):
        
        for node_id in self.root.xpath('/graph/node/@id'):
            

            xpath_expr = '/graph/node[@id="%s"]' % node_id
            
            if len(self.root.xpath(xpath_expr + '/sources/sourceProduct')) != 0:
                print(self.root.xpath(xpath_expr + '/sources/sourceProduct'))[0].attrib['refid']
                print node_id
            else:
                print node_id
        return True
    
    def run(self):
        
        fd, path = tempfile.mkstemp()
        
        try:
        
            self.save_graph(filename=path)
            
            #options = ['/opt/snap6/bin/gpt',
            #   '-x',
            #   '-c',
            #   '2048M',
            #   path]
            
            options = ['/workspace/temp/temp/snap6/snap6/bin/gpt',
               '-x',
               '-c',
               '2048M',
               path]

            p = subprocess.Popen(options,
                stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)

            print p.pid
            res, err = p.communicate()
            print res, err
            if p.returncode != 0:
                raise Exception('An error occurred during the execution of gpt (see log)')
            
        except Exception as e:
            with open('stdout.txt', 'wb') as file:
                file.write(res)
                file.close()
            with open('stderr.txt', 'wb') as file:
                file.write(err)
                file.close()
            
            raise
        finally:
            os.remove(path)
        
def get_snap_parameters(operator):
    
    op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

    op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

    return op_params

### 1. Slave preprocessing

In [ ]:
mygraph = GraphProcessor()

#### 1.1. Read s-1 product (slave)

In [ ]:
operator = 'Read'

index = 0
identifier = input_identifiers[index]
    
parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        parameters[param.getName()] = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe')    
    else:
        parameters[param.getName()] = param.getDefaultValue()
        
node_id = 'Read(%s)' % index
    
read_node = node_id
    
print(parameters)
    
mygraph.add_node(node_id, 'Read', parameters, '')

In [ ]:
mygraph.view_graph()

#### 1.2. Apply orbit file

In [ ]:
operator = 'Apply-Orbit-File'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()


In [ ]:
index = 0
source_node = read_node
    
node_id = 'Apply-Orbit-File(%s)' % index
    
orbit_node = node_id
    
mygraph.add_node(node_id, 'Apply-Orbit-File', parameters, source_node)


In [ ]:
mygraph.view_graph()

#### 1.3. TOPSAR split

In [ ]:
operator = 'TOPSAR-Split'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_split_nodes = []

index_node = 0
source_node = orbit_node

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    
    parameters['subswath'] =  subswath
    parameters['selectedPolarisations'] = 'VV'

    node_id = 'TOPSAR-Split(%s)' % str(index)
    
    
    slave_split_nodes.append(node_id)

    
    mygraph.add_node(node_id, operator, parameters, source_node)

In [ ]:
mygraph.view_graph()

#### 1.4. ThermalNoiseRemoval

In [ ]:
operator = 'ThermalNoiseRemoval'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_tr_nodes = []

for index, slave_split_node in enumerate(slave_split_nodes):
    
    node_id = 'ThermalNoiseRemoval(%s)' % index
    
    source_node = slave_split_node
    
    mygraph.add_node(node_id, operator, parameters, source_node)
    
    slave_tr_nodes.append(node_id)
    
    print(node_id)
    
    print(source_node)

In [ ]:
mygraph.view_graph()

#### 1.5. Calibration

In [ ]:
operator = 'Calibration'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'outputSigmaBand':
        parameters[param.getName()] = 'false'
    elif param.getName() == 'outputBetaBand':
        parameters[param.getName()] = 'true'      
    else:
        parameters[param.getName()] = param.getDefaultValue()
        

In [ ]:
slave_cal_nodes = []

for index, slave_tr_node in enumerate(slave_tr_nodes):
    
    node_id = 'Calibration(%s)' % index
    
    source_node = slave_tr_node
    
    mygraph.add_node(node_id, operator, parameters, source_node)
    
    slave_cal_nodes.append(node_id)
    
    print(node_id)
    
    print(source_node)

In [ ]:
mygraph.view_graph()

#### 1.6. TOPSAR-Deburst

In [ ]:
operator = 'TOPSAR-Deburst'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_deb_nodes = []

for index, slave_cal_node in enumerate(slave_cal_nodes):
    
    node_id = 'TOPSAR-Deburst(%s)' % index
    
    source_node = slave_cal_node
    
    mygraph.add_node(node_id, operator, parameters, source_node)
    
    slave_deb_nodes.append(node_id)
    
    print(node_id)
    
    print(source_node)

In [ ]:
mygraph.view_graph()

#### 1.7. TOPSAR-Merge

In [ ]:
operator = 'TOPSAR-Merge'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_merge_nodes = []

node_id = 'TOPSAR-Merge(%s)' % str(0)
source_nodes = []

for index, slave_deb_node in enumerate(slave_deb_nodes):
    
    source_nodes.append(slave_deb_node)
    
mygraph.add_node(node_id, operator, parameters, source_nodes)

slave_merge_node = node_id

print(node_id)
    
print(source_nodes)

In [ ]:
mygraph.view_graph()

#### 1.8. Multilook

In [ ]:
operator = 'Multilook'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'nRgLooks':
        parameters[param.getName()] = '4'
    elif param.getName() == 'outputIntensity':
        parameters[param.getName()] = 'true'
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:
index = 0
    
node_id = 'Multilook(%s)' % index
    
source_node = slave_merge_node
    
mygraph.add_node(node_id, operator, parameters, source_node)
    
slave_ml_node = node_id
    
print(node_id)
    
print(source_node)

In [ ]:
mygraph.view_graph()

#### 1.9. Write

In [ ]:
operator = 'Write'

output_name = 'temp_slave'

parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        
        param_value = output_name
             
    elif param.getName() == 'formatName':
                
        param_value = 'BEAM-DIMAP'
        
    else:
    
        param_value = param.getDefaultValue()
    
    
    print (param.getName(), param_value)
    
    parameters[param.getName()] = param_value

In [ ]:
mygraph.add_node(operator, 
             operator, 
             parameters,
             slave_ml_node)

In [ ]:
mygraph.view_graph()

In [ ]:
#mygraph.run()

#### Read the products

In [ ]:
operator = 'Read'

read_nodes = []

for index, identifier in enumerate(input_identifiers):
    
    parameters = dict()

    for param in get_snap_parameters(operator):
    
        if param.getName() == 'file':
            parameters[param.getName()] = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe')    
        else:
            parameters[param.getName()] = param.getDefaultValue()
    node_id = 'Read(%s)' % index
    
    read_nodes.append(node_id)
    
    print(parameters)
    
    mygraph.add_node(node_id, 'Read', parameters, '')

In [ ]:
mygraph.view_graph()

### Apply orbit file

In [ ]:
operator = 'Apply-Orbit-File'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()


In [ ]:
orbit_nodes = []

for index, source_node in enumerate(read_nodes):
    
    node_id = 'Apply-Orbit-File(%s)' % index
    
    orbit_nodes.append(node_id)
    
    mygraph.add_node(node_id, 'Apply-Orbit-File', parameters, source_node)
    
print('-----')
print(mygraph)

In [ ]:
#mygraph.view_graph()

### TOPSAR split

In [ ]:
operator = 'TOPSAR-Split'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_split_nodes = []
master_split_nodes = []

for index_node, source_node in enumerate(orbit_nodes):
    
    for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    
        parameters['subswath'] =  subswath
        parameters['selectedPolarisations'] = 'VV'

        node_id = 'TOPSAR-Split(%s)' % str(index_node * 3 + index)
    
        if index_node == 0:
            slave_split_nodes.append(node_id)
        else:
            master_split_nodes.append(node_id)
    
        mygraph.add_node(node_id, operator, parameters, source_node)

In [ ]:
#mygraph.view_graph()

### ThermalNoiseRemoval

In [ ]:
operator = 'ThermalNoiseRemoval'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_tr_nodes = []
master_tr_nodes = []

for index, slave_split_node in enumerate(slave_split_nodes):
    
    node_id = 'ThermalNoiseRemoval(%s)' % index
    
    source_nodes = slave_split_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    slave_tr_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)
    


for index, master_split_node in enumerate(master_split_nodes):
    
    node_id = 'ThermalNoiseRemoval(%s)' % str(index + 3)
    
    source_nodes = master_split_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    master_tr_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)
    


In [ ]:
#mygraph.view_graph()

### Calibration

In [ ]:
operator = 'Calibration'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'outputSigmaBand':
        parameters[param.getName()] = 'false'
    elif param.getName() == 'outputBetaBand':
        parameters[param.getName()] = 'true'      
    else:
        parameters[param.getName()] = param.getDefaultValue()
        

In [ ]:
slave_cal_nodes = []
master_cal_nodes = []

for index, slave_tr_node in enumerate(slave_tr_nodes):
    
    node_id = 'Calibration(%s)' % index
    
    source_nodes = slave_tr_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    slave_cal_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)
    


for index, master_tr_node in enumerate(master_tr_nodes):
    
    node_id = 'Calibration(%s)' % str(index + 3)
    
    source_nodes = master_tr_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    master_cal_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)

In [ ]:
#mygraph.view_graph()

### TOPSAR-Deburst

In [ ]:
operator = 'TOPSAR-Deburst'

In [ ]:
parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_deb_nodes = []
master_deb_nodes = []

for index, slave_cal_node in enumerate(slave_cal_nodes):
    
    node_id = 'TOPSAR-Deburst(%s)' % index
    
    source_nodes = slave_cal_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    slave_deb_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)
    


for index, master_cal_node in enumerate(master_cal_nodes):
    
    node_id = 'TOPSAR-Deburst(%s)' % str(index + 3)
    
    source_nodes = master_cal_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    master_deb_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)

In [ ]:
#mygraph.view_graph()

### TOPSAR-Merge

In [ ]:
operator = 'TOPSAR-Merge'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_merge_nodes = []
master_merge_nodes = []


node_id = 'TOPSAR-Merge(%s)' % str(0)
source_nodes = []
for index, slave_deb_node in enumerate(slave_deb_nodes):
    
    source_nodes.append(slave_deb_nodes[index])
    
mygraph.add_node(node_id, operator, parameters, source_nodes)

slave_merge_nodes.append(node_id)

print(node_id)
    
print(source_nodes)
    

node_id = 'TOPSAR-Merge(%s)' % str(1)
source_nodes = []
for index, master_deb_node in enumerate(master_deb_nodes):
    
    source_nodes.append(master_deb_nodes[index])
    
mygraph.add_node(node_id, operator, parameters, source_nodes)

master_merge_nodes.append(node_id)

print(node_id)
    
print(source_nodes)



In [ ]:
#mygraph.view_graph()

### Multilook

In [ ]:
operator = 'Multilook'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'nRgLooks':
        parameters[param.getName()] = '4'
    elif param.getName() == 'outputIntensity':
        parameters[param.getName()] = 'true'
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:

slave_ml_nodes = []
master_ml_nodes = []

for index, slave_merge_node in enumerate(slave_merge_nodes):
    
    node_id = 'Multilook(%s)' % index
    
    source_nodes = slave_merge_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    slave_ml_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)
    


for index, master_merge_node in enumerate(master_merge_nodes):
    
    node_id = 'Multilook(%s)' % str(index + 1)
    
    source_nodes = master_merge_nodes[index]
    
    mygraph.add_node(node_id, operator, parameters, source_nodes)
    
    master_ml_nodes.append(node_id)
    
    print(node_id)
    
    print(source_nodes)

In [ ]:
#mygraph.view_graph()

### DEM-Assisted-Coregistration

In [ ]:
# TODO: look at resamplingType
operator = 'DEM-Assisted-Coregistration'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'resamplingType':
        parameters[param.getName()] = 'NEAREST_NEIGHBOUR'
    else:
        parameters[param.getName()] = param.getDefaultValue()
    
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
cor_nodes = []

node_id = 'DEM-Assisted-Coregistration'

source_nodes = [slave_ml_nodes[0], master_ml_nodes[0]]

mygraph.add_node(node_id, operator, parameters, source_nodes)

cor_nodes.append(node_id)

print(node_id)
    
print(source_nodes)

In [ ]:
#mygraph.view_graph()

### Speckle-Filter

In [ ]:
operator = 'Speckle-Filter'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'filter':
        parameters[param.getName()] = 'Frost'
    elif param.getName() == 'filterSizeX':
        parameters[param.getName()] = '5'
    elif param.getName() == 'filterSizeY':
        parameters[param.getName()] = '5'
    elif param.getName() == 'dampingFactor':
        parameters[param.getName()] = '2'
    elif param.getName() == 'estimateENL':
        parameters[param.getName()] = 'true'
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:
fil_nodes = []

node_id = 'Speckle-Filter'

source_nodes = cor_nodes[0]

mygraph.add_node(node_id, operator, parameters, source_nodes)

fil_nodes.append(node_id)

print(node_id)
    
print(source_nodes)

In [ ]:
#mygraph.view_graph()

### BandMaths

In [ ]:
operator = 'BandMaths'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'targetBandDescriptors':
        pass
    else:
        parameters[param.getName()] = param.getDefaultValue()
    
targetbandsdic = {'targetBand': {'name': 'dif', 'type': 'float32', 'expression': 'log(Beta0_VV_mst_07Nov2016 / Beta0_VV_slv1_26Oct2016)', 'description': None, 'unit': None, 'noDataValue': '0.0'}}


parameters['targetBands'] = targetbandsdic

parameters

In [ ]:
math_nodes = []

node_id = 'BandMaths'

source_nodes = fil_nodes[0]

mygraph.add_node(node_id, operator, parameters, source_nodes)

math_nodes.append(node_id)

print(node_id)
    
print(source_nodes)

In [ ]:
#mygraph.view_graph()

### Ellipsoid-Correction-RD

In [ ]:
operator = 'Ellipsoid-Correction-RD'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'pixelSpacingInMeter':
        parameters[param.getName()] = '14.71'
    elif param.getName() == 'pixelSpacingInDegree':
        parameters[param.getName()] = '1.321421782939816E-4'
    #elif param.getName() == 'mapProjection':
    #    parameters[param.getName()] = "GEOGCS[&quot;WGS84(DD)&quot;, &#xd;DATUM[&quot;WGS84&quot;, &#xd;SPHEROID[&quot;WGS84&quot;, 6378137.0, 298.257223563]], &#xd;PRIMEM[&quot;Greenwich&quot;, 0.0], &#xd;UNIT[&quot;degree&quot;, 0.017453292519943295], &#xd;AXIS[&quot;Geodetic longitude&quot;, EAST], &#xd;AXIS[&quot;Geodetic latitude&quot;, NORTH]]"
    else:
        parameters[param.getName()] = param.getDefaultValue()


In [ ]:
ellip_nodes = []

node_id = 'Ellipsoid-Correction-RD'

source_nodes = math_nodes[0]

mygraph.add_node(node_id, operator, parameters, source_nodes)

ellip_nodes.append(node_id)

print(node_id)
    
print(source_nodes)

In [ ]:
#mygraph.view_graph()

In [ ]:
ciop = cioppy.Cioppy()

search = ciop.search(end_point=input_references[0],
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,swathIdentifier,wrsLongitudeGrid',
                     model='EOP') 

search2 = ciop.search(end_point=input_references[1],
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,swathIdentifier,wrsLongitudeGrid',
                     model='EOP')

if (search[0]['startdate'] < search2[0]['startdate']):
    start_date = search[0]['startdate']
else:
    start_date = search2[0]['startdate']

if (search[0]['enddate'] > search2[0]['enddate']):
    end_date = search[0]['enddate']
else:
    end_date = search2[0]['enddate']

In [ ]:
output_name = 'S1_IFG_%s_%s_%s' % ("%03d"%int(search[0]['wrsLongitudeGrid']), 
                                    parser.parse(start_date).strftime('%Y%m%d_%H%M%S'),
                                    parser.parse(end_date).strftime('%Y%m%d_%H%M%S'))

In [ ]:
operator = 'Write'
 
parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        
        param_value = output_name
             
    elif param.getName() == 'formatName':
                
        param_value = 'BEAM-DIMAP'
        
    else:
    
        param_value = param.getDefaultValue()
    
    
    print param.getName(), param_value
    
    parameters[param.getName()] = param_value



In [ ]:
mygraph.add_node(operator, 
             operator, 
             parameters,
             ellip_nodes[0])

In [ ]:
mygraph.view_graph()

In [ ]:
mygraph.run()

In [ ]:
reader = ProductIO.getProductReader("BEAM-DIMAP")

ifg = reader.readProductNodes(output_name + '.dim', None)

In [ ]:
band_names = ifg.getBandNames()
target_bands = list(band_names) 
target_bands.append(target_bands[0].replace('i_', 'phase_'))

In [ ]:
BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(target_bands))

for index, band in enumerate(target_bands):
     
    targetBand = BandDescriptor()
    
    if 'phase_' in band:
       
        targetBand.expression = 'atan2(%s, %s)' % (target_bands[1], target_bands[0])
   
    else:
    
        targetBand.expression = band

    targetBand.name = band
    targetBand.type = 'float32'
    
    targetBands[index]= targetBand
        

parameters = HashMap()
parameters.put('targetBands', targetBands)

result = GPF.createProduct('BandMaths', parameters, ifg)

In [ ]:
ProductIO.writeProduct(result, 
                       output_name + '.tif',
                       'GeoTIFF-BigTiff')

In [ ]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % sar)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    polarisation_channels = etree.SubElement(acquisition, '{%s}polarisationChannels' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    #orbit_number.text = metadata['orbitNumber']
    wrs_longitude_grid.text = metadata['wrsLongitudeGrid']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

In [ ]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

In [ ]:
search[0]['identifier'] = output_name
search[0]['wkt'] = result_wkt

search[0]['startdate'] = start_date
search[0]['enddate'] = end_date

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(eop_metadata(search[0]))

In [ ]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (start_date, end_date))
        file.write('geometry=%s' % (search[0]['wkt']))

#### Clean-up

In [ ]:
shutil.rmtree(output_name + '.data')
os.remove(output_name + '.dim')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.